In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Plotting Daily Volume

In [ ]:
#Reading and merging 2 files
df1 = pd.read_csv("/kaggle/input/supply-chain-cel-dataset/canceled_test.csv")
df2 = pd.read_csv("/kaggle/input/supply-chain-cel-dataset/sales_test.csv")
lst = [df2,df1]
df = pd.concat(lst)

In [ ]:
#Checking for null values in the dataframe
sns.heatmap(df.isnull())

In [ ]:
#Converting the 'DATE' column in the standard date-time format and sorting
df['DATE'] = pd.to_datetime(df['DATE'])
df.sort_values('DATE')

In [ ]:
#Grouping by date and reseting index to make column name as 'DATE' again
item_plot = df.groupby('DATE').sum()
item_plot = item_plot.reset_index()

In [ ]:
#Constructing the 'VOLUME' column and correcting for some rows - 
#where orders are cancelled without having a sale (manual/software error)
item_plot['VOLUME'] = item_plot['NS_ORDER'] - item_plot['NC_ORDER']
Vol = np.array(item_plot['VOLUME'])
Vol[Vol<0] = 0
item_plot['VOLUME'] = Vol
item_plot.head()

In [ ]:
#Plotting the daily Volume
plt.figure(figsize=(12,7))
plt.bar(item_plot['DATE'],item_plot['VOLUME']) # A bar chart
plt.xlabel('Date')
plt.ylabel('Volume')

# ABC Analysis

In [ ]:
#ABC analysis per SKU
#Reading and merging 2 files
df1 = pd.read_csv("/kaggle/input/supply-chain-cel-dataset/canceled_test.csv")
df2 = pd.read_csv("/kaggle/input/supply-chain-cel-dataset/sales_test.csv")
lst = [df2,df1]
df = pd.concat(lst)
df

In [ ]:
#Grouping by ITEM No.
df = df.groupby('ITEM').sum()


#Constructing the 'VOLUME' column and correcting for some rows where orders are cancelled without having a sale (manual/software error)
df['VOLUME'] = df['NS_ORDER'] - df['NC_ORDER']
Vol = np.array(df['VOLUME'])
Vol[Vol<0] = 0
df['VOLUME'] = Vol

In [ ]:
#Sorting and constructing column for cumulative values
df = df.sort_values('VOLUME',ascending=False)
df['CUMULATIVE'] = df['VOLUME'].cumsum(axis = 0)
df = df.reset_index()
df

In [ ]:
#ABC Analysis
x = df['CUMULATIVE'].iloc[-1]

def ABC_category(value):
    if value > 0 and value < 0.8*x:
        return 'A'
    elif value >= 0.8*x and value < 0.95*x:
        return 'B'
    else:
        return 'C'
    
df['CATEGORY'] = df['CUMULATIVE'].apply(ABC_category)
df

In [ ]:
#Values per category
df['CATEGORY'].value_counts()

**The 19 items in category 'A' make up for 80% of our sales and we should focus on them to increase profits**

In [ ]:
#Finding the most Important items which make up 80% of the sales volume
df.loc[df['CATEGORY'] == 'A']

In [ ]:
#ABC Analysis per Customer Id
df1 = pd.read_csv("/kaggle/input/supply-chain-cel-dataset/canceled_test.csv")
df2 = pd.read_csv("/kaggle/input/supply-chain-cel-dataset/sales_test.csv")
lst = [df2,df1]
df3 = pd.concat(lst)
CN=df3.groupby("CUSTOMER_NO").sum()
CN['VOLUME']=CN['NS_SHIP']-CN['NC_SHIP']

In [ ]:
Vol = np.array(CN['VOLUME'])
Vol[Vol<0] = 0
CN['VOLUME'] = Vol

In [ ]:
CN = CN.sort_values('VOLUME',ascending=False).reset_index()
CN['CUMULATIVE'] = CN['VOLUME'].cumsum(axis = 0)
CN.head()

In [ ]:
x =CN['CUMULATIVE'].iloc[-1]

def ABC_category(value):
    if value > 0 and value < 0.8*x:
        return 'A'
    elif value >= 0.8*x and value < 0.95*x:
        return 'B'
    else:
        return 'C'

CN['CATEGORY'] = CN['CUMULATIVE'].apply(ABC_category)

In [ ]:
CN['CATEGORY'].value_counts()

**The 62 customers in category 'A' drive 80% of our sales and we should improve service levels for these customers to increase profits.**

In [ ]:
CN.loc[CN['CATEGORY'] == 'A']

# Service Level

In [ ]:
#Service Level per sku - determined by items ordered/items actually shipped

df.drop(['ORDER_NO','LINE','CUSTOMER_NO','NC_ORDER','NC_SHIP','NS_SHIP'], axis='columns', inplace=True)
df['SERVICE LEVEL'] = df['VOLUME']/df['NS_ORDER']

In [ ]:
df['SERVICE LEVEL'] = df['SERVICE LEVEL'].fillna(0)
df.head()

In [ ]:
#Service level per customer 
CN['SERVICE LEVEL'] = CN["VOLUME"]/CN["NS_ORDER"]
CN.drop(['ORDER_NO','LINE','NS_ORDER','NC_ORDER','NC_SHIP','NS_SHIP','ITEM'], axis='columns', inplace=True)
CN['SERVICE LEVEL'] = CN['SERVICE LEVEL'].fillna(0)
CN.head()